В этом файлике изучение onnx-файлов, в дальнейшем планируется адаптировать библиотеку для работы с ними

# Тесты



## Nanodet-детектор

### Информация о модели

In [2]:
import onnx

model_path = 'nanodet_plus_62_192.onnx'
model = onnx.load(model_path)

onnx.checker.check_model(model)  # Проверяем модель на корректность
print(f"Model has {len(model.graph.input)} inputs and {len(model.graph.output)} outputs.")

Model has 1 inputs and 1 outputs.


In [3]:
for input_tensor in model.graph.input:
    print(f"Input name: {input_tensor.name}")
    print(f"Input shape: {input_tensor.type.tensor_type.shape}")
    print(f"Input data type: {input_tensor.type.tensor_type.elem_type}")

Input name: data
Input shape: dim {
  dim_value: 1
}
dim {
  dim_value: 3
}
dim {
  dim_value: 416
}
dim {
  dim_value: 416
}

Input data type: 1


In [4]:
for output_tensor in model.graph.output:
    print(f"Output name: {output_tensor.name}")
    print(f"Output shape: {output_tensor.type.tensor_type.shape}")
    print(f"Output data type: {output_tensor.type.tensor_type.elem_type}")

Output name: output
Output shape: dim {
  dim_value: 1
}
dim {
  dim_value: 3598
}
dim {
  dim_value: 94
}

Output data type: 1


#### Первичные выводы

Модель принимает всё вполне логично - изображение в формате (3, 416, 416)

А вот выдаёт что-то непонятное - (1, 3598, 94). Быть может, это изображение M*N, и для каждого пикселя вероятность принадлежности к одному из 94 классов? Посмотрим, какие могут быть M и N.

In [8]:
from functools import reduce

def factors(n):
    return set(reduce(
        list.__add__,
        ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

display(factors(3598))

{1, 2, 7, 14, 257, 514, 1799, 3598}

Едва ли, поскольку тогда изображение бы было 7x514 или 14x257

Это могло бы быть 3598 найденных объектов, хотя странно, что количество объектов фиксированное, да и что значат 94 признака в таком случае так же неясно. Ведь модель должна быть детектором, то есть возвращать координаты углов и степень уверенности, как это делает изначальный nanodet.

Кстати об изначальном [nanodet](https://github.com/RangiLyu/nanodet?ysclid=mcxcw43rre725659323), вот как он работает:
- возвращает словарь.
- на первом уровне там изображения
- для каждого изображения есть словарь с ключами 0..79, ключ - номер класса.
- внутри содержатся массивы из 5 элементов - координаты четырех углов и уверенность

То есть shape результата оригинального nanodet для одного изображения бы был (1, 79, x, 5), где x - количество найденных объектов для i-го класса

### Исследование конкретных предсказаний модели

Посмотрим, как выглядят предсказания модели для конкретного объекта

In [2]:
import numpy as np
import onnxruntime as ort
import cv2

model_path = 'nanodet_plus_62_192.onnx'
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape

img = cv2.imread('testimg.jpg').astype(np.float32) / 255.0
img = cv2.resize(img, (input_shape[2],input_shape[3])).transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)

pred = session.run(None, {input_name: img})
pred[0].shape # (1, 3598, 94)

(1, 3598, 94)

In [3]:
prediction = pred[0]
prediction[0][0]

array([-7.162573  , -7.633009  , -6.9549394 , -6.7796087 , -6.8664594 ,
       -6.9555717 , -7.705238  , -7.0522127 , -6.64586   , -6.885081  ,
       -6.9224095 , -7.4138317 , -6.6935625 , -7.042231  , -6.4374423 ,
       -7.481862  , -7.052189  , -7.132023  , -7.0860085 , -6.313938  ,
       -7.280868  , -6.3987503 , -6.5284295 , -6.595943  , -6.519201  ,
       -6.785137  , -7.203917  , -7.0519457 , -6.8831525 , -7.0770893 ,
       -6.6204333 , -6.863559  , -7.1004205 , -6.3609414 , -6.724513  ,
       -7.042854  , -7.500639  , -7.2029953 , -6.731427  , -6.8650293 ,
       -6.2754393 , -6.21283   , -7.2643843 , -6.584785  , -7.3847322 ,
       -7.1197557 , -7.7141156 , -6.334044  , -6.488992  , -7.050642  ,
       -6.973954  , -6.98145   , -6.568897  , -8.401864  , -6.8272295 ,
       -7.382326  , -6.8288865 , -6.854281  , -6.7770104 , -6.7170777 ,
       -6.244225  , -7.230572  ,  0.37940204, -0.75468206, -0.49067935,
       -0.69336957, -0.7335961 , -0.7304902 , -0.39784607,  1.99

In [26]:
data = prediction[0][0]
print("Среднее значение:", np.mean(data))
print("Дисперсия:", np.var(data))
print("Минимальное значение:", np.min(data))
print("Максимальное значение:", np.max(data))

Среднее значение: -4.6960826
Дисперсия: 10.080989
Минимальное значение: -8.401864
Максимальное значение: 2.4940338


Что-то странное и на вероятности не похоже, если только они зачем-то не были нормализованы в [-10; 10] или какой-то похожий отрезок

Наконец, попробуем зарендерить изображение как будто оно действительно содержит координаты углов и уверенности в первых 5 значениях. Особо на успех не надеемся, но мало ли

In [11]:
from PIL import Image

# Загрузка и предобработка изображения
img = cv2.imread('testimg.jpg').astype(np.float32) / 255.0
input_height = input_shape[2]  # высота входного изображения
input_width = input_shape[3]   # ширина входного изображения
resized_img = cv2.resize(img, (input_width, input_height))  # сохраняем для визуализации
img = resized_img.transpose((2, 0, 1))  # (channels, height, width)
img = np.expand_dims(img, axis=0)       # (1, channels, height, width)

# Получение предсказаний
pred = session.run(None, {input_name: img})[0]  # (1, 3598, 94)

# Обработка предсказаний
threshold = 0  # порог уверенности
pred = pred[0]   # (3598, 94)

for i in range(pred.shape[0]):
    objectness = pred[i, 4]  # уверенность в наличии объекта
    if objectness > threshold:
        # Извлечение координат bounding box (нормализованные)
        bbox = pred[i, 0:4]  # [center_x, center_y, width, height]
        class_probs = pred[i, 5:]  # вероятности классов
        class_id = np.argmax(class_probs)  # индекс класса с максимальной вероятностью
        score = objectness * class_probs[class_id]  # итоговый скор

        # Преобразование в пиксельные координаты
        center_x = bbox[0] * input_width
        center_y = bbox[1] * input_height
        w = bbox[2] * input_width
        h = bbox[3] * input_height
        x_left = int(center_x - w / 2)
        y_top = int(center_y - h / 2)
        x_right = int(center_x + w / 2)
        y_bottom = int(center_y + h / 2)

        # Отрисовка прямоугольника
        cv2.rectangle(resized_img, (x_left, y_top), (x_right, y_bottom), (0, 255, 0), 2)

        # Добавление подписи
        label = f"Class: {class_id}, Score: {score:.2f}"
        cv2.putText(resized_img, label, (x_left, y_top - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# Сохранение результата
#Image.fromarray(resized_img).show()
#cv2.imwrite('output_with_predictions.jpg', resized_img)
cv2.imwrite('output_with_predictions.jpg', resized_img*255)

True

![](./output_with_predictions.jpg)

Bounding box-ов не видно, так что да, там действительно содержится что-то другое

## Mobilenet

В названии папки и модели есть nsfw, так что, видимо, модель так или иначе связана с детекцией неподобающего контента/классификации изображения на подобающее и неподобающее?? Сейчас проверим

### Информация и модели

In [1]:
import onnx

model_path = 'nsfw_mobilenet2_224.onnx'
model = onnx.load(model_path)

onnx.checker.check_model(model)  # Проверяем модель на корректность
print(f"Model has {len(model.graph.input)} inputs and {len(model.graph.output)} outputs.")

Model has 1 inputs and 1 outputs.


In [2]:
for input_tensor in model.graph.input:
    print(f"Input name: {input_tensor.name}")
    print(f"Input shape: {input_tensor.type.tensor_type.shape}")
    print(f"Input data type: {input_tensor.type.tensor_type.elem_type}")

Input name: input
Input shape: dim {
  dim_param: "unk__610"
}
dim {
  dim_value: 224
}
dim {
  dim_value: 224
}
dim {
  dim_value: 3
}

Input data type: 1


In [3]:
for output_tensor in model.graph.output:
    print(f"Output name: {output_tensor.name}")
    print(f"Output shape: {output_tensor.type.tensor_type.shape}")
    print(f"Output data type: {output_tensor.type.tensor_type.elem_type}")

Output name: prediction
Output shape: dim {
  dim_param: "unk__611"
}
dim {
  dim_value: 5
}

Output data type: 1


В отличие от предыдущей модели, какой-то адекватный вывод - быть может, вероятности пяти классов неподобающего контента? Сейчас проверим на обычном подобающем изображении, если вероятности будут малые, то похоже на правду

### Тесты на изображении

In [7]:
import numpy as np
import onnxruntime as ort
import cv2

model_path = 'nsfw_mobilenet2_224.onnx'
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape

img = cv2.imread('testimg.jpg').astype(np.float32) / 255.0
img = cv2.resize(img, (input_shape[1],input_shape[2]))
img = np.expand_dims(img, axis=0)

pred = session.run(None, {input_name: img})
pred[0].shape

(1, 5)

In [9]:
pred[0]

array([[0.08540819, 0.00759313, 0.8781167 , 0.00566378, 0.02321817]],
      dtype=float32)

Вот использованное изображение:

![](testimg.jpg)

Любопытно, выводы действительно выглядят как вероятности, причем действительно вероятности всех классов, кроме 2, крайне малы.

А вот что модель уверена, что она нашла класс 2 на изображении знака стоп это любопытно, поскольку, насколько я слышал, nsfw он не является.

# Общие выводы

На созвоне надо узнать:

- почему же предоставленные модели на yolo, как было заявлено изначально
- как интерпретировать вывод первой модели (nanodet)
- чему соответствуют классы вывода у второй модели (mobilenet nsfw)